## Table of Contents

1. [Introduction](#1.-introduction)

2. [Imports](#2.-imports)

3. [Daily Return Preparation](#3.-daily-return-preparation)

   3.1 [Load Input CSV files](#3.1-load-input-csv-files)

   3.2 [Clean & Standardize Price Data](#3.2-clean-&-standardize-price-data)

   3.3 [Compute Daily Returns $R_i$, $log(R_i)$, $R_m$, $log(R_m)$, and $R_f$](#3.3-compute-daily-returns-r_i-logr_i-r_m-logr_m-and-r_f)

   3.4 [Align and Merge $R_i$](#3.4-align-and-merge-r_i)

   3.5 [Save Outputs](#3.5-save-outputs)

4. [Portfolio Construction](#4.-portfolio-construction)

   4.1 [Construct Equal-Weighted Portfolio](#4.1-construct-equal-weighted-portfolio)

   4.2 [Construct Optimal Portfolio (Tangency Portfolio)](#4.2-construct-optimal-portfolio-(tangency-portfolio))

5. [Hypothesis Tests](#5.-hypothesis-tests)

   5.1 [Align Series for Hypothesis Tests](#5.1-align-series-for-hypothesis-tests)

   5.2 [Conduct Two-sided F-test for Portfolios Variances](#5.2-conduct-two-sided-f-test-for-portfolios-variances)

6. [Excess Returns for Regression Analysis](#6.-excess-returns-for-regression-analysis)

7. [Market Model Regressions](#7.-market-model-regressions)
   
   7.1 [Regress Excess Stock Returns ~ Excess Market Returns](#7.1-regress-excess-stock-returns-~-excess-market-returns)

   7.2 [7.2 Save Anova Results as DataFrame](#7.2-save-anova-results-as-dataframe)

   7.3 [7.3 Save Regression Results as CSV](#7.3-save-regression-results-as-csv)

### 1. Introduction

### 2. Imports

In [779]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import f
from scipy.stats import ttest_ind
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

### 3. Daily Return Preparation

##### 3.1 Load Input CSV files

In [780]:
# Individual stocks
tickers = ["jpm", "bac", "wfc", "gs", "blk", "pld", "amt", "spg", "avb", "o"]
stock_data = {}

for t in tickers:
    stock_data[t.upper()] = pd.read_csv(f"../data/in/{t}.csv")
    print(t.upper(), stock_data[t.upper()].shape)

# Market index
sp500 = pd.read_csv("../data/in/sp500.csv")
print("\nS&P 500 shape:", sp500.shape)

# Risk-free rate
tbill = pd.read_csv("../data/in/13w_tbill.csv")
print("\nT-bill shape:", tbill.shape)

JPM (1256, 6)
BAC (1256, 6)
WFC (1256, 6)
GS (1256, 6)
BLK (1256, 6)
PLD (1256, 6)
AMT (1256, 6)
SPG (1256, 6)
AVB (1256, 6)
O (1256, 6)

S&P 500 shape: (1255, 6)

T-bill shape: (1256, 6)


##### 3.2 Clean & Standardize Price Data

In [781]:
# Clean individual stock price files
clean_stocks = {}

for t in tickers:
    df = stock_data[t.upper()].copy()

    # Convert Date to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Clean "Close/Last" (remove any "$" symbols, convert to float)
    df['Close'] = df['Close/Last'].replace('[\\$,]', '', regex=True).astype(float)

    # Keep only Date + Close columns
    df = df[['Date', 'Close']].sort_values('Date')

    clean_stocks[t.upper()] = df

    print(f"Sample cleaned stock ({t.upper()}):")
    print(clean_stocks[t.upper()].head(3))

# Clean S&P 500
sp500_clean = sp500.copy()
sp500_clean['Date'] = pd.to_datetime(sp500_clean['Date'], format='%m/%d/%y')
sp500_clean = sp500_clean[['Date', 'Close']].sort_values('Date')

# Clean 13-week T-bill
tbill_clean = tbill.copy()
tbill_clean['Date'] = pd.to_datetime(tbill_clean['Date'], format='%m/%d/%y')
tbill_clean = tbill_clean[['Date', 'Close']].sort_values('Date')

# Show first 3 rows from each for verification
print("Sample cleaned stock (JPM):")
print(clean_stocks['JPM'].head(3))

print("\nSample cleaned S&P500:")
print(sp500_clean.head(3))

print("\nSample cleaned T-bill:")
print(tbill_clean.head(3))

Sample cleaned stock (JPM):
           Date   Close
1255 2020-10-19   99.80
1254 2020-10-20  100.37
1253 2020-10-21   99.37
Sample cleaned stock (BAC):
           Date  Close
1255 2020-10-19  23.72
1254 2020-10-20  24.14
1253 2020-10-21  24.05
Sample cleaned stock (WFC):
           Date  Close
1255 2020-10-19  22.54
1254 2020-10-20  22.81
1253 2020-10-21  22.70
Sample cleaned stock (GS):
           Date   Close
1255 2020-10-19  205.69
1254 2020-10-20  208.03
1253 2020-10-21  202.91
Sample cleaned stock (BLK):
           Date   Close
1255 2020-10-19  634.89
1254 2020-10-20  645.07
1253 2020-10-21  634.99
Sample cleaned stock (PLD):
           Date   Close
1255 2020-10-19  103.02
1254 2020-10-20  103.53
1253 2020-10-21  102.65
Sample cleaned stock (AMT):
           Date   Close
1255 2020-10-19  239.60
1254 2020-10-20  243.80
1253 2020-10-21  241.74
Sample cleaned stock (SPG):
           Date  Close
1255 2020-10-19  63.91
1254 2020-10-20  64.46
1253 2020-10-21  64.21
Sample cleaned stock 

##### 3.3 Compute Daily Returns $R_i$, $log(R_i)$, $R_m$, $log(R_m)$, and $R_f$

In [782]:
# Compute daily returns (Ret) and log returns (LogRet) for each stock
stock_returns = {}

for t in tickers:
    df = clean_stocks[t.upper()].copy()

    # Arithmetic return
    df['Ret'] = df['Close'] / df['Close'].shift(1) - 1

    # Log return
    df['LogRet'] = np.log(df['Close'] / df['Close'].shift(1))

    df = df.dropna()
    stock_returns[t.upper()] = df[['Date', 'Ret', 'LogRet']]
    
    print(f"Sample Returns (Ri = {t.upper()}):")
    print(stock_returns[t.upper()].head(3))

# Compute market returns and log returns
sp500_ret = sp500_clean.copy()
sp500_ret['Market_Ret'] = sp500_ret['Close'] / sp500_ret['Close'].shift(1) - 1
sp500_ret['Market_LogRet'] = np.log(sp500_ret['Close'] / sp500_ret['Close'].shift(1))
sp500_ret = sp500_ret.dropna()
sp500_ret = sp500_ret[['Date', 'Market_Ret', 'Market_LogRet']]

print(f"Sample Market Returns (Rm):")
print(sp500_ret.head(3))

# Convert T-bill annualized yield to daily rate (RF)
tbill_daily = tbill_clean.copy()
tbill_daily['RF_Daily'] = (1 + tbill_daily['Close']/100)**(1/252) - 1
tbill_daily = tbill_daily[['Date', 'RF_Daily']]

print(f"Sample Daily Risk-Free Rate (Rf):")
print(tbill_daily.head(3))

Sample Returns (Ri = JPM):
           Date       Ret    LogRet
1254 2020-10-20  0.005711  0.005695
1253 2020-10-21 -0.009963 -0.010013
1252 2020-10-22  0.035323  0.034713
Sample Returns (Ri = BAC):
           Date       Ret    LogRet
1254 2020-10-20  0.017707  0.017552
1253 2020-10-21 -0.003728 -0.003735
1252 2020-10-22  0.034096  0.033527
Sample Returns (Ri = WFC):
           Date       Ret    LogRet
1254 2020-10-20  0.011979  0.011908
1253 2020-10-21 -0.004822 -0.004834
1252 2020-10-22  0.024229  0.023940
Sample Returns (Ri = GS):
           Date       Ret    LogRet
1254 2020-10-20  0.011376  0.011312
1253 2020-10-21 -0.024612 -0.024920
1252 2020-10-22  0.012271  0.012197
Sample Returns (Ri = BLK):
           Date       Ret    LogRet
1254 2020-10-20  0.016034  0.015907
1253 2020-10-21 -0.015626 -0.015750
1252 2020-10-22  0.000299  0.000299
Sample Returns (Ri = PLD):
           Date       Ret    LogRet
1254 2020-10-20  0.004950  0.004938
1253 2020-10-21 -0.008500 -0.008536
1252 2020-1

##### 3.4 Align and Merge $R_i$

In [783]:
# Merge all stock returns into wide format

merged_stocks = None

for t in tickers:
    df = stock_returns[t.upper()].copy()
    df = df.rename(columns={
        'Ret': f"{t.upper()}_Ret",
        'LogRet': f"{t.upper()}_LogRet"
    })

    if merged_stocks is None:
        merged_stocks = df
    else:
        merged_stocks = pd.merge(merged_stocks, df, on='Date', how='inner')

print("Merged Daily Stock Returns (wide format):")
print(merged_stocks.head(3))

Merged Daily Stock Returns (wide format):
        Date   JPM_Ret  JPM_LogRet   BAC_Ret  BAC_LogRet   WFC_Ret  \
0 2020-10-20  0.005711    0.005695  0.017707    0.017552  0.011979   
1 2020-10-21 -0.009963   -0.010013 -0.003728   -0.003735 -0.004822   
2 2020-10-22  0.035323    0.034713  0.034096    0.033527  0.024229   

   WFC_LogRet    GS_Ret  GS_LogRet   BLK_Ret  ...   PLD_Ret  PLD_LogRet  \
0    0.011908  0.011376   0.011312  0.016034  ...  0.004950    0.004938   
1   -0.004834 -0.024612  -0.024920 -0.015626  ... -0.008500   -0.008536   
2    0.023940  0.012271   0.012197  0.000299  ...  0.001461    0.001460   

    AMT_Ret  AMT_LogRet   SPG_Ret  SPG_LogRet   AVB_Ret  AVB_LogRet     O_Ret  \
0  0.017529    0.017377  0.008606    0.008569  0.000963    0.000962  0.005957   
1 -0.008450   -0.008485 -0.003878   -0.003886  0.002886    0.002881  0.009814   
2 -0.024696   -0.025006  0.033328    0.032785  0.003494    0.003488  0.006870   

   O_LogRet  
0  0.005940  
1  0.009766  
2  0.0068

##### 3.5 Save Outputs

In [784]:
# Save wide stock returns
merged_stocks.to_csv("../data/out/3_daily_stock_returns.csv", index=False)
print("Saved: 3_daily_stock_returns.csv")

# Save market returns
daily_market_returns = sp500_ret.copy()
daily_market_returns.to_csv("../data/out/3_daily_market_returns.csv", index=False)
print("Saved: 3_daily_market_returns.csv")

# Save risk-free returns
daily_riskfree_returns = tbill_daily.copy()
daily_riskfree_returns.to_csv("../data/out/3_daily_riskfree_returns.csv", index=False)
print("Saved: 3_daily_riskfree_returns.csv")

Saved: 3_daily_stock_returns.csv
Saved: 3_daily_market_returns.csv
Saved: 3_daily_riskfree_returns.csv


### 4. Portfolio Construction

##### 4.1 Construct Equal-Weighted Portfolio

In [785]:
# 4.1.1 Extract Daily Log Returns
logret_cols = [c for c in merged_stocks.columns if c.endswith("_LogRet")]
logret_df = merged_stocks[["Date"] + logret_cols].copy()

# 4.1.2 Compute Equal-Weighted Log Return Series
n_stocks = len(logret_cols)
ew_weights = np.full(n_stocks, 1/n_stocks)

logret_matrix = logret_df[logret_cols].values
logret_df["EW_LogRet"] = logret_matrix.dot(ew_weights)

# 4.1.3 Save Equal-Weighted Portfolio Output
ew_output = logret_df[["Date", "EW_LogRet"]].copy()
ew_output.to_csv("../data/out/4_portfolio_equal_weight_logreturns.csv", index=False)
print("Saved: 4_portfolio_equal_weight_logreturns.csv")

Saved: 4_portfolio_equal_weight_logreturns.csv


##### 4.2 Construct Optimal Portfolio (Tangency Portfolio)

In [786]:
# 4.2.1 Extract Daily Log Returns
logret_cols = [c for c in merged_stocks.columns if c.endswith("_LogRet")]
logret_df = merged_stocks[["Date"] + logret_cols].copy()

# Drop Date for matrix calculations
R = logret_df[logret_cols].values

# 4.2.2 Compute μ (mean vector) and Σ (covariance matrix)
mu = R.mean(axis=0)
Sigma = np.cov(R, rowvar=False)

# 4.2.3 Align Risk-Free Rate (RF_Daily)
rf_df = daily_riskfree_returns.copy()
rf_df = rf_df.sort_values("Date")

# Inner join to ensure perfect alignment with stock return dates
aligned = pd.merge(logret_df[["Date"]], rf_df, on="Date", how="inner")
# rf = aligned["RF_Daily"].values

# 4.2.4 Define Negative Sharpe Ratio Objective (Project 1 pattern)
def neg_sharpe(weights, mu, Sigma, rf_daily):
    port_ret = np.dot(weights, mu)
    port_var = np.dot(weights.T, np.dot(Sigma, weights))
    port_vol = np.sqrt(port_var)
    return -(port_ret - rf_daily) / port_vol

# 4.2.5 Set Constraints and Bounds, Then Solve with SLSQP
n = len(mu)
initial_w = np.full(n, 1/n)
bounds = [(0, 1)] * n
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

rf_daily = aligned["RF_Daily"].mean()

opt_result = minimize(
    neg_sharpe,
    initial_w,
    args=(mu, Sigma, rf_daily),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints
)

optimal_w = opt_result.x

# 4.2.6 Save Optimal Weights
w_df = pd.DataFrame({
    "Ticker": logret_cols,
    "Weight": optimal_w
})
print(f"Weights for optimal portfolio: \n{w_df.sort_values(by='Weight', ascending=False).round(4)}")
w_df.to_csv("../data/out/4_optimal_weights_daily.csv", index=False)
print("Saved: 4_optimal_weights_daily.csv")

# 4.2.7 Construct Optimal-Weighted Portfolio Log Return Series
logret_matrix = logret_df[logret_cols].values
logret_df["OPT_LogRet"] = logret_matrix.dot(optimal_w)

# 4.2.8 Save Optimal Portfolio Output
opt_output = logret_df[["Date", "OPT_LogRet"]].copy()
opt_output.to_csv("../data/out/4_portfolio_optimal_weight_logreturns.csv", index=False)
print("Saved: 4_portfolio_optimal_weight_logreturns.csv")

Weights for optimal portfolio: 
       Ticker  Weight
3   GS_LogRet  0.5764
0  JPM_LogRet  0.1796
2  WFC_LogRet  0.1600
7  SPG_LogRet  0.0840
1  BAC_LogRet  0.0000
9    O_LogRet  0.0000
5  PLD_LogRet  0.0000
4  BLK_LogRet  0.0000
6  AMT_LogRet  0.0000
8  AVB_LogRet  0.0000
Saved: 4_optimal_weights_daily.csv
Saved: 4_portfolio_optimal_weight_logreturns.csv


### 5. Hypothesis Tests

##### 5.1 Align Series for Hypothesis Tests

In [787]:
# 5.1.1: Align by Date (inner join)
df = ew_output.merge(opt_output, on="Date", how="inner")

# Extract aligned numpy vectors
ew = df["EW_LogRet"].values
opt = df["OPT_LogRet"].values

print("Aligned shape:", df.shape)
print("EW observations lost:", len(ew_output) - len(df))
print("OPT observations lost:", len(opt_output) - len(df))

Aligned shape: (1255, 3)
EW observations lost: 0
OPT observations lost: 0


##### 5.2 Conduct two-sided F-test for Portfolios Variances

$H_0: \sigma_{EW}^2 = \sigma_{Opt}^2$<br>
$H_1: \sigma_{EW}^2 \neq \sigma_{Opt}^2$

In [788]:
# Sample variances
sew_sq = np.var(ew, ddof=1)     # EW portfolio variance
sopt_sq = np.var(opt, ddof=1)    # OPT portfolio variance

# F statistic (larger variance on top to keep F ≥ 1)
if sew_sq >= sopt_sq:
    F_stat = sew_sq / sopt_sq
    df1 = len(ew) - 1
    df2 = len(opt) - 1
else:
    F_stat = sopt_sq / sew_sq
    df1 = len(opt) - 1
    df2 = len(ew) - 1

# Two-sided p-value
p_value = 2 * min(
    f.cdf(F_stat, df1, df2),
    1 - f.cdf(F_stat, df1, df2)
)

print("F-statistic:", F_stat)
print("p-value:", p_value)

# Conclusion
if p_value < 0.05:
    print("Reject the null hypothesis: Variances differ.")
else:
    print("Fail to reject the null hypothesis: Variances are statistically equal.")

F-statistic: 1.6769232302440173
p-value: 2.220446049250313e-16
Reject the null hypothesis: Variances differ.


In [789]:
# Additional F-Test Analysis

# One-sided p-values
p_right = 1 - f.cdf(F_stat, df1, df2)   # H1: σ_EW² > σ_OPT²
p_left  = f.cdf(F_stat, df1, df2)       # H1: σ_EW² < σ_OPT²

print("Right-tail p-value (EW > OPT):", p_right)
print("Left-tail p-value  (EW < OPT):", p_left)

# 95% Confidence Interval for the Variance Ratio
alpha = 0.05

F_lower = f.ppf(alpha/2, df1, df2)
F_upper = f.ppf(1 - alpha/2, df1, df2)

CI_lower = F_stat / F_upper
CI_upper = F_stat / F_lower

print("\n95% Confidence Interval for variance ratio (σ_EW² / σ_OPT²):")
print("Lower bound:", CI_lower)
print("Upper bound:", CI_upper)


Right-tail p-value (EW > OPT): 1.1102230246251565e-16
Left-tail p-value  (EW < OPT): 0.9999999999999999

95% Confidence Interval for variance ratio (σ_EW² / σ_OPT²):
Lower bound: 1.501125299430942
Upper bound: 1.8733089910602738


##### 5.3 Conduct Welch Two-Sample t-Test for Portfolio Means

$H_0: \mu_{EW} = \mu_{Opt}$<br>
$H_1: \mu_{EW} \neq \mu_{Opt}$

In [790]:
# Welch t-test (unequal variances)

t_stat, p_value = ttest_ind(ew, opt, equal_var=False)

print("t-statistic:", t_stat)
print("p-value:", p_value)

# Conclusion
if p_value < 0.05:
    print("Reject the null hypothesis: Means differ.")
else:
    print("Fail to reject the null hypothesis: Means are statistically equal.")

t-statistic: -0.8443775792667289
p-value: 0.39854413075045547
Fail to reject the null hypothesis: Means are statistically equal.


### 6. Excess Returns for Regression Analysis

In [791]:
# Align by Date (inner join across stocks, market, and risk-free)
aligned = (
    merged_stocks
        .merge(daily_market_returns, on="Date", how="inner")
        .merge(daily_riskfree_returns, on="Date", how="inner")
)

# 6.2 Compute Excess Returns for each stock
excess_returns = aligned[["Date"]].copy()

for col in merged_stocks.columns:
    if col.endswith("_Ret") and not col.startswith("Market"):
        ticker = col.replace("_Ret", "")
        excess_returns[f"{ticker}_Excess"] = aligned[col] - aligned["RF_Daily"]

# 6.3 Compute Market Excess Return
excess_returns["Market_Excess"] = aligned["Market_Ret"] - aligned["RF_Daily"]

print("Aligned shape:", excess_returns.shape)
print("Daily stock returns observations lost:", len(merged_stocks) - len(excess_returns))
print("Market returns observations lost:", len(daily_market_returns) - len(excess_returns))
print("Risk-free returns observations lost:", len(daily_riskfree_returns) - len(excess_returns))
print(f"\n\nSample Excess Returns:\n\n {excess_returns.head(3)}")

Aligned shape: (1250, 12)
Daily stock returns observations lost: 5
Market returns observations lost: 4
Risk-free returns observations lost: 6


Sample Excess Returns:

         Date  JPM_Excess  BAC_Excess  WFC_Excess  GS_Excess  BLK_Excess  \
0 2020-10-27   -0.018870   -0.028121   -0.038770  -0.027149   -0.012315   
1 2020-10-28   -0.028092   -0.015936   -0.029334  -0.030206   -0.022492   
2 2020-10-29    0.006523    0.002979   -0.001892   0.000893    0.002694   

   PLD_Excess  AMT_Excess  SPG_Excess  AVB_Excess  O_Excess  Market_Excess  
0   -0.013241   -0.004052   -0.041748   -0.024414 -0.018388      -0.003029  
1   -0.016445   -0.025537   -0.019687   -0.051262 -0.015669      -0.035291  
2    0.027379    0.006602    0.043611    0.025767  0.012278       0.011944  


### 7. Market Model Regressions

$R_i−R_f=β_0+β_1 (R_m−R_f)+𝜀_i$

##### 7.1 Regress Excess Stock Returns ~ Excess Market Returns

In [792]:
def run_market_model(df, y_col):
    formula = f"{y_col} ~ Market_Excess"
    model = smf.ols(formula=formula, data=df).fit()
    anova_table = anova_lm(model)
    residuals = model.resid
    return model, anova_table, residuals

In [793]:
tickers = [
    "JPM", "BAC", "WFC", "GS", "BLK",
    "PLD", "AMT", "SPG", "AVB", "O"
]

results_dict = {}
anova_tables = {}
residuals_df = pd.DataFrame({"Date": excess_returns["Date"]})

summary = []

for t in tickers:
    y_col = f"{t}_Excess"

    model, anova_table, residuals = run_market_model(excess_returns, y_col)

    results_dict[t] = model
    anova_tables[t] = anova_table
    residuals_df[f"residual_{t}"] = residuals

    summary.append({
        "ticker": t,
        "R2": model.rsquared,
        "beta0": model.params["Intercept"],
        "se_beta0": model.bse["Intercept"],
        "t_beta0": model.tvalues["Intercept"],
        "p_beta0": model.pvalues["Intercept"],
        "beta1": model.params["Market_Excess"],
        "se_beta1": model.bse["Market_Excess"],
        "t_beta1": model.tvalues["Market_Excess"],
        "p_beta1": model.pvalues["Market_Excess"],
        "ci_lower_beta1": model.conf_int().loc["Market_Excess", 0],
        "ci_upper_beta1": model.conf_int().loc["Market_Excess", 1],
        "anova_F": anova_table.loc["Market_Excess", "F"],
        "anova_p": anova_table.loc["Market_Excess", "PR(>F)"],
    })

    print(f"\n\nRegression Results for {t}")
    print(f"\n{model.summary()}\n")
    print(f"{'='*90}\n{model.summary2().tables[1]}\n{'='*90}\n")

residuals_df = residuals_df.sort_values("Date")
regression = pd.DataFrame(summary)



Regression Results for JPM

                            OLS Regression Results                            
Dep. Variable:             JPM_Excess   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     750.0
Date:                Tue, 25 Nov 2025   Prob (F-statistic):          1.08e-129
Time:                        01:09:35   Log-Likelihood:                 3694.0
No. Observations:                1250   AIC:                            -7384.
Df Residuals:                    1248   BIC:                            -7374.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       

##### 7.2 Save Anova Results as DataFrame

In [794]:
# Convert anova_tables dict into a long, tidy ANOVA summary DataFrame
anova_summary_rows = []

for ticker, table in anova_tables.items():
    # Extract only the Market_Excess row
    row = table.loc["Market_Excess"]

    anova_summary_rows.append({
        "ticker": ticker,
        "degrees_of_freedom": int(row["df"]),
        "sum_sq": row["sum_sq"],
        "mean_sq": row["sum_sq"] / int(row["df"]),
        "F": row["F"],
        "p_value": row["PR(>F)"],
    })

anova_summary = pd.DataFrame(anova_summary_rows)

##### 7.3 Save Regression Results as CSV

In [795]:
# Merge regression_summary and anova_summary on ticker
regression_summary = regression.merge(
    anova_summary,
    on="ticker",
    how="inner",
    suffixes=("", "_anova")   # in case needed
)
regression_summary = regression_summary.drop(["F", "p_value"], axis=1)
regression_summary.to_csv("../data/out/7_market_model_regression_summary.csv", index=False)
print("Saved: 7_market_model_regression_summary.csv")

Saved: 7_market_model_regression_summary.csv


In [796]:
print(regression_summary.head())

  ticker        R2     beta0  se_beta0   t_beta0   p_beta0     beta1  \
0    JPM  0.375372  0.000437  0.000357  1.225095  0.220771  0.906879   
1    BAC  0.358127  0.000157  0.000404  0.387883  0.698169  0.988184   
2    WFC  0.306636  0.000630  0.000472  1.333478  0.182618  1.029374   
3     GS  0.457038  0.000565  0.000363  1.555490  0.120083  1.091759   
4    BLK  0.612271 -0.000058  0.000293 -0.197809  0.843227  1.206019   

   se_beta1    t_beta1        p_beta1  ci_lower_beta1  ci_upper_beta1  \
0  0.033115  27.385928  1.080791e-129        0.841912        0.971846   
1  0.037449  26.387670  2.657389e-122        0.914715        1.061653   
2  0.043816  23.492964  2.342349e-101        0.943413        1.115336   
3  0.033684  32.411442  1.046414e-167        1.025674        1.157843   
4  0.027167  44.393092  5.028077e-259        1.152722        1.259317   

       anova_F        anova_p  degrees_of_freedom    sum_sq   mean_sq  
0   749.989060  1.080791e-129                   1  0.119